In [2]:
import nba_api.stats.endpoints
import requests
import json
import pandas as pd
import nba_api

/Users/jackborman/PycharmProjects/NBA/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [39]:
data_trad_season = nba_api.stats.endpoints.LeagueDashPlayerStats(
    # last_n_games=None,
    measure_type_detailed_defense='Base',
    pace_adjust='N',
    per_mode_detailed='PerGame',
    season='2024-25',
    season_type_all_star='Regular Season'
    ).get_data_frames()[0]

data_trad_playoffs = nba_api.stats.endpoints.LeagueDashPlayerStats(
    # last_n_games=None,
    measure_type_detailed_defense='Base',
    pace_adjust='N',
    per_mode_detailed='PerGame',
    season='2024-25',
    season_type_all_star='Playoffs'
    ).get_data_frames()[0]

data_trad_playoffs_total = nba_api.stats.endpoints.LeagueDashPlayerStats(
    # last_n_games=None,
    measure_type_detailed_defense='Base',
    pace_adjust='N',
    per_mode_detailed='Totals',
    season='2024-25',
    season_type_all_star='Playoffs'
    ).get_data_frames()[0]

rs_trad_data = data_trad_season[data_trad_season['TEAM_ABBREVIATION'].isin(['MIN', 'OKC'])]
p_trad_data = data_trad_playoffs[data_trad_playoffs['TEAM_ABBREVIATION'].isin(['MIN', 'OKC'])]
data_trad_playoffs_total = data_trad_playoffs_total[data_trad_playoffs_total['TEAM_ABBREVIATION'].isin(['MIN', 'OKC'])].rename(columns={
    'MIN':'PLAYOFF_MIN',
    'PLAYER_ID': 'PLAYER_ID', 'PLAYER_NAME': 'NAME', 'TEAM_ABBREVIATION': 'TEAM',
})[['PLAYER_ID', 'NAME', 'TEAM','PLAYOFF_MIN']]

p_trad_data_total = data_trad_playoffs_total[~data_trad_playoffs_total['PLAYOFF_MIN'].isna()]

p_trad_data_total

,PLAYER_ID,NAME,TEAM,PLAYOFF_MIN
4,1630598,Aaron Wiggins,OKC,154.796667
5,1642349,Ajay Mitchell,OKC,38.241667
8,1627936,Alex Caruso,OKC,247.245000
16,1630162,Anthony Edwards,MIN,399.000000
37,1641717,Cason Wallace,OKC,230.803333
38,1631096,Chet Holmgren,OKC,331.600000
58,1641794,Dillon Jones,OKC,21.053333
60,1628978,Donte DiVincenzo,MIN,258.133333
78,1628392,Isaiah Hartenstein,OKC,284.125000
79,1630198,Isaiah Joe,OKC,121.768333


In [45]:
fg3_comp = pd.merge(
    rs_trad_data[['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION', 'MIN', 'PTS', 'FG3M', 'FG3A', 'FG3_PCT', 'FGA']],
    p_trad_data[['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION','MIN', 'PTS', 'FG3M', 'FG3A', 'FG3_PCT', 'FGA']], 
    on=['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION'], how='left'
).rename(columns={
    'PLAYER_ID': 'PLAYER_ID', 'PLAYER_NAME': 'NAME', 'TEAM_ABBREVIATION': 'TEAM', 
    'MIN_x':'MIN_RS', 'MIN_y':'MIN_P', 
    'PTS_x':'PTS_RS', 'PTS_y':'PTS_P',
    'FG3M_x':'FG3M_RS', 'FG3M_y':'FG3M_P',
    'FG3A_x':'FG3A_RS', 'FG3A_y':'FG3A_P',
    'FG3_PCT_x':'FG3_PCT_RS', 'FG3_PCT_y':'FG3_PCT_P',
    'FGA_x':'FGA_RS', 'FGA_y':'FGA_P'
}).sort_values(by=['MIN_P'], ascending=False)[[
    'PLAYER_ID', 'NAME', 'TEAM', 
    'MIN_RS', 'MIN_P', 
    # 'PTS_RS', 'PTS_P',
    'FG3M_RS', 'FG3M_P',
    'FG3A_RS', 'FG3A_P',
    'FG3_PCT_RS', 'FG3_PCT_P', 
    'FGA_RS', 'FGA_P'
]].reset_index(drop=True)

fg3_comp = fg3_comp[~fg3_comp['MIN_P'].isna()].assign(
    FG3_Rate_RS=round(fg3_comp['FG3A_RS']/fg3_comp['FGA_RS'], 3),
    FG3_Rate_P=round(fg3_comp['FG3A_P']/fg3_comp['FGA_P'], 3), 
).assign(
    FG3_Rate_X = lambda x:x['FG3_Rate_P'] - x['FG3_Rate_RS'],
    FG3_PCT_X=round(fg3_comp['FG3_PCT_P']-fg3_comp['FG3_PCT_RS'], 3), 
)

fg3_comp = pd.merge(
    fg3_comp, 
    p_trad_data_total[['PLAYER_ID', 'NAME', 'TEAM','PLAYOFF_MIN']],
    on=['PLAYER_ID', 'NAME', 'TEAM'], how='left'
)

fg3_comp[fg3_comp['PLAYOFF_MIN'] >= 100]

,PLAYER_ID,NAME,TEAM,MIN_RS,MIN_P,FG3M_RS,FG3M_P,FG3A_RS,FG3A_P,FG3_PCT_RS,FG3_PCT_P,FGA_RS,FGA_P,FG3_Rate_RS,FG3_Rate_P,FG3_Rate_X,FG3_PCT_X,PLAYOFF_MIN
0,1630162,Anthony Edwards,MIN,36.3,39.9,4.1,3.5,10.3,9.1,0.395,0.385,20.4,21.1,0.505,0.431,-0.074,-0.010,399.000000
1,203944,Julius Randle,MIN,32.3,37.6,1.6,1.9,4.6,5.5,0.344,0.345,13.6,16.9,0.338,0.325,-0.013,0.001,376.141667
2,1628983,Shai Gilgeous-Alexander,OKC,34.2,36.6,2.1,1.5,5.7,5.3,0.375,0.293,21.8,20.9,0.261,0.254,-0.007,-0.082,403.001667
3,1630183,Jaden McDaniels,MIN,31.9,36.0,1.2,1.0,3.7,2.9,0.330,0.345,10.2,11.2,0.363,0.259,-0.104,0.015,360.466667
4,1631114,Jalen Williams,OKC,32.4,35.0,1.8,1.4,4.9,5.4,0.365,0.254,16.9,16.7,0.290,0.323,0.033,-0.111,384.953333
5,1631096,Chet Holmgren,OKC,27.4,30.1,1.4,1.5,3.6,4.8,0.379,0.321,10.7,11.8,0.336,0.407,0.071,-0.058,331.600000
6,203497,Rudy Gobert,MIN,33.2,28.0,0.0,0.0,0.0,0.0,0.000,0.000,7.1,5.6,0.000,0.000,0.000,0.000,280.280000
7,1629652,Luguentz Dort,OKC,29.2,26.7,2.4,1.9,5.8,6.8,0.412,0.280,8.4,8.0,0.690,0.850,0.160,-0.132,293.938333
8,1628392,Isaiah Hartenstein,OKC,27.9,25.8,0.0,0.0,0.3,0.0,0.000,0.000,8.4,7.8,0.036,0.000,-0.036,0.000,284.125000
9,1628978,Donte DiVincenzo,MIN,25.9,25.8,2.8,1.4,7.1,5.6,0.397,0.250,9.6,8.7,0.740,0.644,-0.096,-0.147,258.133333


In [5]:
# data_trad_season = nba_api.stats.endpoints.LeagueDashPlayerStats(
#     # last_n_games=None,
#     measure_type_detailed_defense='Base',
#     pace_adjust='N',
#     per_mode_detailed='PerGame',
#     season='2024-25',
#     season_type_all_star='Regular Season'
#     ).get_data_frames()[0]

g1_track_v3 = nba_api.stats.endpoints.BoxScorePlayerTrackV3(
    game_id='0042400311'
).get_data_frames()[0]

g1_track_v3

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,contestedFieldGoalsMade,contestedFieldGoalsAttempted,contestedFieldGoalPercentage,uncontestedFieldGoalsMade,uncontestedFieldGoalsAttempted,uncontestedFieldGoalsPercentage,fieldGoalPercentage,defendedAtRimFieldGoalsMade,defendedAtRimFieldGoalsAttempted,defendedAtRimFieldGoalPercentage
0,0042400311,1610612750,Minnesota,Timberwolves,MIN,timberwolves,1630183,Jaden,McDaniels,J. McDaniels,...,0,0,0.000,3,5,0.600,0.599,3,4,0.750
1,0042400311,1610612750,Minnesota,Timberwolves,MIN,timberwolves,203944,Julius,Randle,J. Randle,...,1,2,0.500,8,11,0.727,0.691,3,4,0.750
2,0042400311,1610612750,Minnesota,Timberwolves,MIN,timberwolves,203497,Rudy,Gobert,R. Gobert,...,1,3,0.333,0,0,0.000,0.333,2,4,0.500
3,0042400311,1610612750,Minnesota,Timberwolves,MIN,timberwolves,1630162,Anthony,Edwards,A. Edwards,...,2,2,1.000,3,11,0.273,0.385,1,3,0.333
4,0042400311,1610612750,Minnesota,Timberwolves,MIN,timberwolves,201144,Mike,Conley,M. Conley,...,0,2,0.000,2,6,0.333,0.250,1,3,0.333
5,0042400311,1610612750,Minnesota,Timberwolves,MIN,timberwolves,1629675,Naz,Reid,N. Reid,...,1,4,0.250,0,7,0.000,0.090,4,5,0.800
6,0042400311,1610612750,Minnesota,Timberwolves,MIN,timberwolves,1628978,Donte,DiVincenzo,D. DiVincenzo,...,0,2,0.000,3,12,0.250,0.213,1,3,0.333
7,0042400311,1610612750,Minnesota,Timberwolves,MIN,timberwolves,1629638,Nickeil,Alexander-Walker,N. Alexander-Walker,...,1,1,1.000,2,10,0.200,0.273,0,1,0.000
8,0042400311,1610612750,Minnesota,Timberwolves,MIN,timberwolves,1641740,Jaylen,Clark,J. Clark,...,1,1,1.000,0,1,0.000,0.500,0,0,0.000
9,0042400311,1610612750,Minnesota,Timberwolves,MIN,timberwolves,1630545,Terrence,Shannon Jr.,T. Shannon Jr.,...,0,0,0.000,1,2,0.500,0.500,0,0,0.000


In [32]:
randle_passing = nba_api.stats.endpoints.PlayerDashPtPass(
    season='2024-25',
    season_type_all_star='Playoffs', 
    per_mode_simple='PerGame',
    player_id='203944',
    team_id='1610612750'
).get_data_frames()[0]

randle_passing

,PLAYER_ID,PLAYER_NAME_LAST_FIRST,TEAM_NAME,TEAM_ID,TEAM_ABBREVIATION,PASS_TYPE,G,PASS_TO,PASS_TEAMMATE_PLAYER_ID,FREQUENCY,...,AST,FGM,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3A,FG3_PCT
0,203944,"Randle, Julius",Minnesota Timberwolves,1610612750,MIN,made,11,"Shannon Jr., Terrence",1630545,0.002,...,0.00,0.00,0.09,0.000,0.00,0.09,0.000,0.00,0.00,NaN
1,203944,"Randle, Julius",Minnesota Timberwolves,1610612750,MIN,made,11,"McDaniels, Jaden",1630183,0.141,...,0.91,1.00,3.00,0.333,0.64,1.82,0.350,0.36,1.18,0.308
2,203944,"Randle, Julius",Minnesota Timberwolves,1610612750,MIN,made,11,"Edwards, Anthony",1630162,0.274,...,1.45,2.55,5.73,0.444,1.55,3.36,0.459,1.00,2.36,0.423
3,203944,"Randle, Julius",Minnesota Timberwolves,1610612750,MIN,made,11,"Reid, Naz",1629675,0.083,...,1.00,1.00,1.73,0.579,0.36,0.64,0.571,0.64,1.09,0.583
4,203944,"Randle, Julius",Minnesota Timberwolves,1610612750,MIN,made,11,"Alexander-Walker, Nickeil",1629638,0.100,...,0.64,0.73,1.82,0.400,0.36,0.64,0.571,0.36,1.18,0.308
5,203944,"Randle, Julius",Minnesota Timberwolves,1610612750,MIN,made,11,"DiVincenzo, Donte",1628978,0.158,...,0.45,0.55,2.82,0.194,0.00,0.64,0.000,0.55,2.18,0.250
6,203944,"Randle, Julius",Minnesota Timberwolves,1610612750,MIN,made,11,"Gobert, Rudy",203497,0.029,...,0.36,0.45,0.73,0.625,0.45,0.73,0.625,0.00,0.00,NaN
7,203944,"Randle, Julius",Minnesota Timberwolves,1610612750,MIN,made,11,"Conley, Mike",201144,0.214,...,0.64,0.64,2.64,0.241,0.27,1.00,0.273,0.36,1.64,0.222
